In [1]:
from ast import Index
from json import load
import os
import getpass
import time
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4

from pinecone import Pinecone, ServerlessSpec


if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = "recipes"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)
index = pc.Index(index_name)

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()

loader = CSVLoader ("./recipes.csv")

docs = loader.load_and_split(text_splitter=splitter)
doc_ids = [f"doc-{i}" for i in range(len(docs))]
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

vector_store = PineconeVectorStore(
    
    embedding=embeddings,
    index_name=index_name,
)
uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)


/Users/bryan/Documents/GitHub/gpt/env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


['b2d20700-ddd9-4623-99c8-cc7017c94c9d',
 'c3462c8e-9a69-419e-964d-55ef3358539d',
 'baef1614-65fe-4d92-bd99-d1a055817d89',
 '735718cc-a859-4a79-b12b-0bb70b916bea',
 'a37424a8-bb63-4df2-9b0f-6a16eef31e8c',
 '77413737-e183-459b-b6c7-ad95b6d4376e',
 '44b6a639-f0aa-420a-bfb3-4a0414950ea0',
 '042bcc79-6a6e-46c7-8c91-63515ee0a551',
 '2b7937a8-58ea-4863-97ec-23e86ecbb2db',
 'cfe9c5db-89a7-4dfd-8c87-0b42edcb0934',
 '4bd73a25-b0e6-4f7d-8241-9b68c85b963e',
 '5a114cd0-c04b-4773-96a3-ebc72521a5e6',
 '503de4d0-e1ed-4ef6-848b-89398e396af7',
 '6c78c036-63e2-4954-9222-1cd33b4173a5',
 '3dedf299-4998-4f98-97e1-24f68c09023e',
 '4d56bb7f-b293-4442-8cc0-02d6a8b781d5',
 'b62c3529-d93f-44b8-a8c8-1d3470c56ef6',
 'a13872a8-eb89-4e04-bf0f-c287f93570ad',
 'f0205fbf-5145-445b-959f-53a1cfe93f06',
 'c77e4bc6-29d1-4463-9a71-f3f6d0e83e92',
 '243f6dd1-a094-4dd1-a592-23b70bdaee94',
 '1848a94b-cb78-457a-9471-bb42ecc0a1e9',
 '6a441077-23db-4508-a849-4d56386836ad',
 '1519e2bf-1b3a-4616-8cad-92fa35b61018',
 'edf32a6f-52e2-